# Implement models with PyTorch to allow for GPU cuda compatability

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [2]:
def train_model(input_name:str, output_name:str) -> None:
    # load data
    df = pd.read_csv(input_name)
    X = df.drop('Outcome', axis=1)
    y = df['Outcome']

    # convert data to numpy
    X = np.array(X)
    y = np.array(y)

    # split and scale the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # convert data to cuda tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).cuda()
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).cuda()
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).cuda()
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).cuda()

    # create batches for the data to reduce noise
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # define model (should probably use command line arguments in the future)
    class ANN_Model(nn.Module):
        def __init__(self, input_size):
            super(ANN_Model, self).__init__()
            self.fc1 = nn.Linear(input_size, 256)
            self.fc2 = nn.Linear(256, 128)
            self.fc3 = nn.Linear(128, 64)
            self.fc4 = nn.Linear(64, 32)
            self.fc5 = nn.Linear(32, 1)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.1)
            
        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.relu(self.fc2(x))
            x = self.dropout(x)
            x = self.relu(self.fc3(x))
            x = self.dropout(x)
            x = self.relu(self.fc4(x))
            x = self.dropout(x)
            x = self.fc5(x)
            return x

    # create an instance of the model
    model = ANN_Model(input_size=X_train.shape[1]).cuda()
    device = torch.device("cuda") # replace with "cpu" if this gives error
    model.to(device)

    # define the loss function and optimizer
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # tain the model
    num_epochs = 15
    for epoch in range(num_epochs):
        model.train()
        for batch_inputs, batch_targets in train_loader:
            # move batch to gpu
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            
            # forward pass
            outputs = model(batch_inputs)
            loss = loss_function(outputs, batch_targets.unsqueeze(1))
            
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        # print status
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

    # print accuracy
    model.eval()
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)
    outputs = model(X_test_tensor)
    predicted_labels = torch.round(torch.sigmoid(outputs))
    accuracy = (predicted_labels == y_test_tensor.unsqueeze(1)).sum().item() / len(y_test_tensor)
    print(f"Accuracy: {accuracy}")

    torch.save(model.state_dict(), output_name)

In [3]:
train_model("csvs/ml_X.csv", "built_models/torch_ml.pth")

Epoch 1/15, Loss: 0.7137329578399658
Epoch 2/15, Loss: 0.6905653476715088
Epoch 3/15, Loss: 0.6487824320793152
Epoch 4/15, Loss: 0.6060296297073364
Epoch 5/15, Loss: 0.5244685411453247
Epoch 6/15, Loss: 0.4980783760547638
Epoch 7/15, Loss: 0.4687689244747162
Epoch 8/15, Loss: 0.4362174868583679
Epoch 9/15, Loss: 0.5765647888183594
Epoch 10/15, Loss: 0.4152742624282837
Epoch 11/15, Loss: 0.5833065509796143
Epoch 12/15, Loss: 0.25526532530784607
Epoch 13/15, Loss: 0.2922390401363373
Epoch 14/15, Loss: 0.40410467982292175
Epoch 15/15, Loss: 0.25608494877815247
Accuracy: 0.8248303043573462


In [4]:
train_model("csvs/ou_X.csv", "built_models/torch_ou.pth")

Epoch 1/15, Loss: 0.6874184012413025
Epoch 2/15, Loss: 0.6766497492790222
Epoch 3/15, Loss: 0.6740106344223022
Epoch 4/15, Loss: 0.6955709457397461
Epoch 5/15, Loss: 0.6873725652694702
Epoch 6/15, Loss: 0.704105794429779
Epoch 7/15, Loss: 0.7020933032035828
Epoch 8/15, Loss: 0.684398353099823
Epoch 9/15, Loss: 0.7021235227584839
Epoch 10/15, Loss: 0.6944047808647156
Epoch 11/15, Loss: 0.6892191171646118
Epoch 12/15, Loss: 0.6837195158004761
Epoch 13/15, Loss: 0.7019463777542114
Epoch 14/15, Loss: 0.6887359619140625
Epoch 15/15, Loss: 0.6758884191513062
Accuracy: 0.5210249671484888


In [5]:
train_model("csvs/pl_X.csv", "built_models/torch_pl.pth")

Epoch 1/15, Loss: 0.8619431257247925
Epoch 2/15, Loss: 0.5576226711273193
Epoch 3/15, Loss: 0.5900101065635681
Epoch 4/15, Loss: 0.6315622925758362
Epoch 5/15, Loss: 0.6693400144577026
Epoch 6/15, Loss: 0.6567090749740601
Epoch 7/15, Loss: 0.5545150637626648
Epoch 8/15, Loss: 0.8072935938835144
Epoch 9/15, Loss: 0.49366122484207153
Epoch 10/15, Loss: 0.6074188947677612
Epoch 11/15, Loss: 0.5078808069229126
Epoch 12/15, Loss: 0.4914930760860443
Epoch 13/15, Loss: 0.6793432235717773
Epoch 14/15, Loss: 0.2949889302253723
Epoch 15/15, Loss: 0.6146039962768555
Accuracy: 0.5774956217162872
